In [2]:
from mysql.connector import connect, Error
import pandas as pd
from faker import Faker
import random

In [3]:
#initialize faker
fake = Faker()

In [4]:
categories = ["Groceries", "stationary", "shopping", "entertainment", "travel", "health", "bills", "rent", "Investments", "other"]
payment_modes = ["Cash", "Debit Card", "UPI", "Netbanking", "Credit Card"]

descriptions = {
    "Groceries": [
        "bought fruits from Mart",
        "bought vegetables from Mart",
        "bought meat from store",
        "bought some households from store"
    ],
    "stationary": [
        "bought pen from store",
        "bought notebook from store",
        "bought pencil from store",
        "bought scissors from store"
    ],
    "shopping": [
        "bought clothes from mall",
        "bought shoes from mall",
        "bought accessories from mall",
        "bought bags from mall"
    ],
    "entertainment": [
        "bought movie tickets",
        "bought concert tickets",
        "bought event tickets",
        "bought party tickets"
    ],
    "travel": [
        "bought flight tickets",
        "bought train tickets",
        "bought bus tickets",
        "bought taxi tickets"
    ],
    "health": [
        "bought medicines",
        "bought supplements",
        "health checkup expenses",
        "bought health insurance"
    ],
    "bills": [
        "paid electricity bill",
        "paid water bill",
        "paid internet bill",
        "paid phone bill"
    ],
    "rent": [
        "paid house rent"
    ],
    "Investments": [
        "bought stocks",
        "bought mutual funds",
        "bought gold",
        "bought silver"
    ],
    "other": [
        "bought gifts",
        "bought gadgets"
    ]
}

In [5]:
connection = connect(
    host="localhost",
    user="root",  # Replace with your MySQL username
    password="Jashu@123",  # Replace with your MySQL password
)
cursor = connection.cursor()

In [6]:
connection.reconnect()

In [64]:
# Create a new database and table for storing expenses
create_db_query = """
CREATE DATABASE IF NOT EXISTS ExpenseDB;
USE ExpenseDB;
CREATE TABLE IF NOT EXISTS Expenses (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Date DATE,
    Category VARCHAR(50),
    Payment_Mode VARCHAR(50),
    Description TEXT,
    Amount DECIMAL(10, 2)
);
"""

# Reconnect to the database if not connected
if not connection.is_connected():
    connection.reconnect()

# Execute the query
cursor.execute(create_db_query, multi=True)


<generator object CMySQLCursor._execute_iter at 0x0000026F51CB7E20>

In [65]:
# Function to generate data for each month
def gen_exp_data_per_month():
    exp_data_per_month = {}
    for month in range(1, 13):
        month_str = f"2025-{month:02d}"
        data = []
        for _ in range(120):
            category = random.choice(categories)
            date = fake.date_this_year(before_today=True, after_today=True)
            while date.month != month:
                date = fake.date_this_year(before_today=True, after_today=True)  # Ensure correct month
            expense = {
                "Date": date,
                "Category": category,
                "Payment_Mode": random.choice(payment_modes),
                "Description": random.choice(descriptions[category]),
                "Amount": random.randint(100, 10000),
            }
            data.append(expense)
        exp_data_per_month[month_str] = pd.DataFrame(data)
    return exp_data_per_month

exp_data_per_month = gen_exp_data_per_month()

In [95]:
# MySQL connection and data insertion
try:
    connection = connect(
        host="localhost",
        user="root",  # Replace with your MySQL username
        password="Jashu@123",  # Replace with your MySQL password
        database="ExpenseDB"
    )
    cursor = connection.cursor()
except Error as e:
    print(f"Error occurred: {e}")

In [67]:
insert_query = """
    INSERT INTO Expenses (Date, Category, Payment_Mode, Description, Amount)
    VALUES (%s, %s, %s, %s, %s)
    """

try:
    for month, df in exp_data_per_month.items():
        if not df.empty:  # Check if DataFrame is not empty
            print(f"Inserting data for month: {month}")
            data_to_insert = df.to_records(index=False).tolist()
            cursor.executemany(insert_query, data_to_insert)  # Insert data in batches
            connection.commit()  # Commit after each month
            print(f"Data for month {month} inserted successfully.")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection closed.")


Inserting data for month: 2025-01
Data for month 2025-01 inserted successfully.
Inserting data for month: 2025-02
Data for month 2025-02 inserted successfully.
Inserting data for month: 2025-03
Data for month 2025-03 inserted successfully.
Inserting data for month: 2025-04
Data for month 2025-04 inserted successfully.
Inserting data for month: 2025-05
Data for month 2025-05 inserted successfully.
Inserting data for month: 2025-06
Data for month 2025-06 inserted successfully.
Inserting data for month: 2025-07
Data for month 2025-07 inserted successfully.
Inserting data for month: 2025-08
Data for month 2025-08 inserted successfully.
Inserting data for month: 2025-09
Data for month 2025-09 inserted successfully.
Inserting data for month: 2025-10
Data for month 2025-10 inserted successfully.
Inserting data for month: 2025-11
Data for month 2025-11 inserted successfully.
Inserting data for month: 2025-12
Data for month 2025-12 inserted successfully.
Connection closed.


In [7]:
# Set Pandas display options to show wide tables in one line
pd.set_option('display.max_columns', None)   # Show all columns
pd.set_option('display.width', 1000)         # Increase display width
pd.set_option('display.colheader_justify', 'left')  # Center align column headers

In [8]:
# Connect to MySQL database
try:
    connection = connect(
        host="localhost",
        user="root",       # Replace with your username
        password="Jashu@123",   # Replace with your password
        database="ExpenseDB"        # Replace with your database name
    )
    if connection.is_connected():
        cursor = connection.cursor()
        print("Successfully connected to the database.")
except Error as e:
    print(f"Error occurred: {e}")

Successfully connected to the database.


In [9]:
# 1. Total Amount Spent in Each Category
query1 = "SELECT Category, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Category ORDER BY Total_Spent DESC;"
cursor.execute(query1)
result1 = cursor.fetchall()
print("1. Total Amount Spent in Each Category:")
print(pd.DataFrame(result1, columns=['Category', 'Total_Spent']))

1. Total Amount Spent in Each Category:
  Category       Total_Spent
0      Groceries  893896.00 
1         travel  728203.00 
2  entertainment  722465.00 
3       shopping  720119.00 
4          bills  706848.00 
5     stationary  702537.00 
6         health  671190.00 
7          other  660460.00 
8    Investments  640509.00 
9           rent  635856.00 


In [74]:
# 2. Monthly Expense Summary
query2 = "SELECT DATE_FORMAT(Date, '%Y-%m') AS Month, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Month ORDER BY Month;"
cursor.execute(query2)
result2 = cursor.fetchall()
print("\n2. Monthly Expense Summary:")
print(pd.DataFrame(result2, columns=['Month', 'Total_Spent']))



2. Monthly Expense Summary:
   Month    Total_Spent
0   2025-01  584672.00 
1   2025-02  581060.00 
2   2025-03  538057.00 
3   2025-04  588217.00 
4   2025-05  658115.00 
5   2025-06  595013.00 
6   2025-07  646903.00 
7   2025-08  515017.00 
8   2025-09  629936.00 
9   2025-10  606460.00 
10  2025-11  555133.00 
11  2025-12  583500.00 


In [75]:
# 3. Top 5 Most Expensive Transactions
query3 = "SELECT * FROM Expenses ORDER BY Amount DESC LIMIT 5;"
cursor.execute(query3)
result3 = cursor.fetchall()
print("Top 5 Most Expensive Transactions:")
print(pd.DataFrame(result3, columns=['id', 'Date', 'Category', 'Payment_Mode', 'Description', 'Amount']))

Top 5 Most Expensive Transactions:
   id   Date        Category    Payment_Mode Description            Amount   
0    33  2025-01-18        rent          UPI        paid house rent  10000.00
1    99  2025-01-29  stationary  Credit Card  bought pen from store   9997.00
2   325  2025-03-07  stationary   Debit Card  bought pen from store   9978.00
3  1143  2025-10-12      travel         Cash     bought bus tickets   9973.00
4  1026  2025-09-12    shopping   Netbanking  bought bags from mall   9962.00


In [76]:
# 4. Average Spending by Payment Mode
query4 = "SELECT Payment_Mode, AVG(Amount) AS Avg_Spending FROM Expenses GROUP BY Payment_Mode;"
cursor.execute(query4)
result4 = cursor.fetchall()
print("\n4. Average Spending by Payment Mode:")
print(pd.DataFrame(result4, columns=['Payment_Mode', 'Avg_Spending']))


4. Average Spending by Payment Mode:
  Payment_Mode Avg_Spending
0   Netbanking  4820.544025
1         Cash  5100.784512
2  Credit Card  4669.921348
3   Debit Card  4967.808429
4          UPI  5019.360269


In [77]:
# 5. Number of Transactions per Category
query5 = "SELECT Category, COUNT(*) AS Num_Transactions FROM Expenses GROUP BY Category ORDER BY Num_Transactions DESC;"
cursor.execute(query5)
result5 = cursor.fetchall()
print("\n5. Number of Transactions per Category:")
print(pd.DataFrame(result5, columns=['Category', 'Num_Transactions']))


5. Number of Transactions per Category:
  Category        Num_Transactions
0      Groceries  179             
1       shopping  150             
2         travel  147             
3          bills  147             
4     stationary  141             
5          other  140             
6    Investments  138             
7  entertainment  138             
8         health  135             
9           rent  125             


In [78]:
# 6. Most Frequently Used Payment Mode
query6 = "SELECT Payment_Mode, COUNT(*) AS Usage_Count FROM Expenses GROUP BY Payment_Mode ORDER BY Usage_Count DESC LIMIT 1;"
cursor.execute(query6)
result6 = cursor.fetchall()
print("\n6. Most Frequently Used Payment Mode:")


6. Most Frequently Used Payment Mode:


In [79]:
# 7. Total Spending for a Specific Month (e.g., December 2025)
query7 = "SELECT SUM(Amount) AS Total_Spent_december FROM Expenses WHERE DATE_FORMAT(Date, '%Y-%m') = '2025-09';"
cursor.execute(query7)
result7 = cursor.fetchall()
print("\n7. Total Spending for december:")
print(pd.DataFrame(result7, columns=['Total_Spent_december']))


7. Total Spending for december:
  Total_Spent_december
0  629936.00          


In [81]:
# 8. Categories with Spending Above a Certain Threshold
query8 = "SELECT Category, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Category HAVING Total_Spent > 50000 ORDER BY Total_Spent DESC;"
cursor.execute(query8)
result8 = cursor.fetchall()
print("\n8. Categories with Spending Above 50,000:")
print(pd.DataFrame(result8, columns=['Category', 'Total_Spent']))


8. Categories with Spending Above 50,000:
  Category       Total_Spent
0      Groceries  893896.00 
1         travel  728203.00 
2  entertainment  722465.00 
3       shopping  720119.00 
4          bills  706848.00 
5     stationary  702537.00 
6         health  671190.00 
7          other  660460.00 
8    Investments  640509.00 
9           rent  635856.00 


In [82]:
# 9. Daily Expense Trend
query9 = "SELECT Date, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Date ORDER BY Date ;"
cursor.execute(query9)
result9 = cursor.fetchall()
print("\n9. Daily Expense Trend:")
print(pd.DataFrame(result9, columns=['Date', 'Total_Spent']))


9. Daily Expense Trend:
    Date        Total_Spent
0    2025-01-01  12235.00  
1    2025-01-02  23685.00  
2    2025-01-03   3161.00  
3    2025-01-04  30281.00  
4    2025-01-05  14074.00  
..          ...         ...
354  2025-12-26  18119.00  
355  2025-12-27   6906.00  
356  2025-12-28  14379.00  
357  2025-12-30  13647.00  
358  2025-12-31  15284.00  

[359 rows x 2 columns]


In [84]:
# 10. Highest Spending Day
query10 = "SELECT Date, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Date ORDER BY Total_Spent DESC LIMIT 1;"
cursor.execute(query10)
result10 = cursor.fetchall()
print("\n10. Highest Spending Day:")
print(pd.DataFrame(result10, columns=['Date', 'Total_Spent']))



10. Highest Spending Day:
  Date        Total_Spent
0  2025-11-06  51499.00  


In [85]:
# 11. Spending Breakdown by Description
query11 = "SELECT Description, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Description ORDER BY Total_Spent DESC;"
cursor.execute(query11)
result11 = cursor.fetchall()
print("\n11. Spending Breakdown by Description:")
print(pd.DataFrame(result11, columns=['Description', 'Total_Spent']))


11. Spending Breakdown by Description:
   Description                        Total_Spent
0                     paid house rent  635856.00 
1                        bought gifts  358804.00 
2                      bought gadgets  301656.00 
3              bought meat from store  234096.00 
4         bought vegetables from Mart  228898.00 
5             bought fruits from Mart  218398.00 
6             health checkup expenses  217191.00 
7   bought some households from store  212504.00 
8          bought scissors from store  209588.00 
9               bought bags from mall  209581.00 
10                    paid phone bill  206743.00 
11                 bought bus tickets  205382.00 
12               bought event tickets  199909.00 
13               bought movie tickets  196248.00 
14           bought clothes from mall  192404.00 
15            bought health insurance  192328.00 
16                      bought stocks  190285.00 
17             bought concert tickets  188650.00 
18        

In [86]:
# 12. Find the Category with the Lowest Spending
query12 = "SELECT Category, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Category ORDER BY Total_Spent ASC LIMIT 1;"
cursor.execute(query12)
result12 = cursor.fetchall()
print("\n12. Category with the Lowest Spending:")
print(pd.DataFrame(result12, columns=['Category', 'Total_Spent']))


12. Category with the Lowest Spending:
  Category Total_Spent
0  rent     635856.00 


In [87]:
# 13. Total Spending by Quarter
query13 = "SELECT QUARTER(Date) AS Quarter, SUM(Amount) AS Total_Spent FROM Expenses GROUP BY Quarter ORDER BY Quarter;"
cursor.execute(query13)
result13 = cursor.fetchall()
print("\n13. Total Spending by Quarter:")
print(pd.DataFrame(result13, columns=['Quarter', 'Total_Spent']))


13. Total Spending by Quarter:
   Quarter Total_Spent
0  1        1703789.00
1  2        1841345.00
2  3        1791856.00
3  4        1745093.00


In [88]:
# 14. Identify the Most Common Description
query14 = "SELECT Description, COUNT(*) AS Occurrences FROM Expenses GROUP BY Description ORDER BY Occurrences DESC LIMIT 1;"
cursor.execute(query14)
result14 = cursor.fetchall()
print("\n14. Most Common Description:")
print(pd.DataFrame(result14, columns=['Description', 'Occurrences']))       


14. Most Common Description:
  Description       Occurrences
0  paid house rent  125        


In [89]:
# 15. Spending Comparison: Cash vs Credit Card
query15 = "SELECT Payment_Mode, SUM(Amount) AS Total_Spent FROM Expenses WHERE Payment_Mode IN ('Cash', 'Credit Card') GROUP BY Payment_Mode;"
cursor.execute(query15)
result15 = cursor.fetchall()
print("\n15. Spending Comparison (Cash vs Credit Card):")
print(pd.DataFrame(result15, columns=['Payment_Mode', 'Total_Spent']))       


15. Spending Comparison (Cash vs Credit Card):
  Payment_Mode Total_Spent
0         Cash  1514933.00
1  Credit Card  1246869.00


In [90]:
# 16. Identify the Least Used Payment Mode
query16 = "SELECT Payment_Mode, COUNT(*) AS Usage_Count FROM Expenses GROUP BY Payment_Mode ORDER BY Usage_Count ASC LIMIT 1;"
cursor.execute(query16)
result16 = cursor.fetchall()
print("\n16. Least Used Payment Mode:")
print(pd.DataFrame(result16, columns=['Payment_Mode', 'Usage_Count']))


16. Least Used Payment Mode:
  Payment_Mode  Usage_Count
0  Debit Card   261        


In [10]:
# 17. Monthly Spending in the 'Travel' Category
query17 = "SELECT DATE_FORMAT(Date, '%Y-%m') AS Month, SUM(Amount) AS Total_Spent FROM Expenses WHERE Category = 'Travel' GROUP BY Month ORDER BY Month;"
cursor.execute(query17)
result17 = cursor.fetchall()
print("\n17. Monthly Spending in the 'Travel' Category:")
print(pd.DataFrame(result17, columns=['Month', 'Total_Spent']))


17. Monthly Spending in the 'Travel' Category:
   Month    Total_Spent
0   2025-01  95392.00  
1   2025-02  52848.00  
2   2025-03  71546.00  
3   2025-04  80537.00  
4   2025-05  59503.00  
5   2025-06  68350.00  
6   2025-07  59726.00  
7   2025-08  57868.00  
8   2025-09  46329.00  
9   2025-10  49288.00  
10  2025-11  34710.00  
11  2025-12  52106.00  


In [92]:
try:
    # Your code here
    pass  # Replace this with the actual code that might raise an error

except Error as e:
    print(f"Error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("\nConnection closed.")


Connection closed.
